In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import json
import csv
import folium 

In [8]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [3]:
#Create a link to MongoDB using MongoClient(The name of new database will be geo_example)
db = MongoClient().geo_example

In [9]:
# Create the database and collection "places" with an geolocation index "loc"
db.busstops.create_index([("latlon", GEO2D)])


u'loc_2d'

In [10]:
df = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Paradas/gtfs_linha438-paradas.csv')
#records = json.loads(df.T.to_json()).values()

for row in df.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    busstops438 = [{ "line": row[1]['linha'],
                     "description":row[1]['descricao'],
                     "agency":row[1]['agencia'],
                     "senquency" : row[1]['sequencia'],
                     "latitude": row[1]['latitude'],
                     "longitude": row [1]['longitude'], 
                     "latlon": [ row[1]['latitude'], row[1]['longitude'] ]
                    }]
    
db.busstops.insert_many(busstops438)
    
    


In [12]:
len(df)

148

In [23]:
mymap=folium.Map(location=[-22.9163, -43.2512], zoom_start=16)

mymap.simple_marker([-22.9163, -43.2512], popup='latitude: -22.9163, longitude: -43.2512')    
mymap.circle_marker(location=[-22.9163, -43.2512], radius=150,line_color='red',
                  fill_color='#3186cc', fill_opacity=0.2)

#para calcular los 2 puntos que mas cerca están de [-22.9163, -43.2512] que es la primera parada de la secuencia
for doc in db.busstops.find({"latlon": {"$near": [-22.9163, -43.2512]}}).limit(2):
    print doc
    mymap.polygon_marker( [ doc['latitude'], doc['longitude'] ], fill_color='#45647d', num_sides=4, radius=6)




{u'latlon': [-22.9163, -43.2512], u'description': u'438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR', u'senquency': 1, u'agency': u'Fetranspor', u'longitude': -43.2512, u'latitude': -22.9163, u'line': 438, u'_id': ObjectId('55f8318e312f910da79a9925')}
{u'latlon': [-22.9177, -43.2512], u'description': u'438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR', u'senquency': 2, u'agency': u'Fetranspor', u'longitude': -43.2512, u'latitude': -22.9177, u'line': 438, u'_id': ObjectId('55f8318e312f910da79a98ba')}


In [25]:
#para pintar el shape de la linea

onibus438_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha438-shapes.csv')

locations438 = []
for row in onibus438_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations438.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

mymap.line(locations438)
embed_map(mymap)

In [27]:
#muestra los registros de los GPS del omnibus 438 con id A27592 desde las 3pm hasta las 8pm
gps08022015 = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/aniversario450Rio_event_analysis/20150208.txt', delimiter = ('\t'))
gps438_A27592 = gps08022015[(gps08022015['line'] == '438' ) & (gps08022015['bus_id'] == 'A27514') & (gps08022015['timestamp'] > '08-02-2015 15:00:00' ) & (gps08022015['timestamp'] < '08-02-2015 20:00:00' )]
 

In [66]:
gps438_A27592_3pm_4pm = gps438_A27592[(gps438_A27592['timestamp'] > '08-02-2015 15:00:00' ) & (gps438_A27592['timestamp'] < '08-02-2015 17:00:00' )]

In [67]:
len(gps438_A27592_3pm_4pm)

108

In [68]:
gps438_A27592_3pm_4pm

,timestamp,bus_id,line,latitude,longitude,speed,other
1633888,08-02-2015 15:00:05,A27514,438,-22.915689,-43.229500,23,37
1636101,08-02-2015 15:00:35,A27514,438,-22.915800,-43.229591,0,121
1640437,08-02-2015 15:02:05,A27514,438,-22.919161,-43.233212,0,80
1643717,08-02-2015 15:03:05,A27514,438,-22.918859,-43.234299,24,72
1646928,08-02-2015 15:04:05,A27514,438,-22.916580,-43.235062,0,61
1653320,08-02-2015 15:06:05,A27514,438,-22.916031,-43.235229,9,40
1655425,08-02-2015 15:06:35,A27514,438,-22.913931,-43.235729,26,64
1658837,08-02-2015 15:07:35,A27514,438,-22.914530,-43.239231,47,104
1661912,08-02-2015 15:08:35,A27514,438,-22.915190,-43.242561,29,94
1664593,08-02-2015 15:09:05,A27514,438,-22.915850,-43.246078,37,113


In [71]:

# Create the database and collection  with an geolocation index "loc"
#db.gps438_A27592_3pm_4pm_table.create_index([("latlon", GEO2D)])
db.mytable1.create_index([("latlon", GEO2D)])

#insertar en mongo el dataframe filtrado
for row in gps438_A27592_3pm_4pm.iterrows():
    myarray = [{ "timestamp":row[1]['timestamp'],
                "bus_id" : row[1]['bus_id'],
                "latitude": row[1]['latitude'],
                "longitude": row [1]['longitude'],
                "speed":row[1]['speed'],
                "other":row[1]['other'],
                "latlon": [ row[1]['latitude'], row[1]['longitude'] ]
                
                  }]
                 
    db.mytable1.insert_many(myarray)



In [83]:
#para calcular los puntos donde los GPS hicieron observaciones y que mas cerca están de [-22.9163, -43.2512] que es la primera parada de la secuencia

for doc in db.mytable1.find({"latlon": {"$near": [-22.9163, -43.2512]}}).limit(50):
#    print doc
    mymap.polygon_marker( [ doc['latitude'], doc['longitude'] ], fill_color='browm', num_sides=4, radius=6)


embed_map(mymap)



In [109]:
#buscar la hora en que llegó al punto (inicial)
mymapnew=folium.Map(location=[-22.9563, -43.2512], zoom_start=12)

#poniendo el popup en el punto inicial
#mymapnew.simple_marker([-22.9882, -43.2283], popup='Punto Inicial, latitude: -22.9882, longitude:-43.2283')
                        
#Punto inicial, radio
mymapnew.circle_marker(location=[-22.9882, -43.2283], radius=1500,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

mymapnew.line(locations438)
embed_map(mymapnew)

In [110]:
#para calcular los puntos donde los GPS hicieron observaciones y que mas cerca están de [-22.9163, -43.2512] que es la primera parada de la secuencia

for doc in db.mytable1.find({"latlon": {"$near": [-22.9882, -43.2283]}}).limit(1):
    print doc
    mymapnew.polygon_marker( [ doc['latitude'], doc['longitude'] ], fill_color='browm', num_sides=4, radius=6)
  
embed_map(mymapnew)

{u'latlon': [-22.98835, -43.22826], u'timestamp': u'08-02-2015 16:44:35', u'longitude': -43.22826, u'other': 74.0, u'latitude': -22.98835, u'_id': ObjectId('55f8614d312f910da79a9afe'), u'bus_id': u'A27514', u'speed': 21.0}


In [116]:
#recorrido gavea-princesa isabel hoarario 4:44 om en adelante
gps438_A27592_4_44pm_adelante = gps438_A27592[(gps438_A27592['timestamp'] > '08-02-2015 16:44:00' ) & (gps438_A27592['timestamp'] < '08-02-2015 17:55:00' )]
gps438_A27592_4_44pm_adelante



,timestamp,bus_id,line,latitude,longitude,speed,other
1970221,08-02-2015 16:44:05,A27514,438,-22.987551,-43.228958,0,55
1972403,08-02-2015 16:44:35,A27514,438,-22.988350,-43.228260,21,74
1975524,08-02-2015 16:45:35,A27514,438,-22.988461,-43.228168,0,112
1978369,08-02-2015 16:46:05,A27514,438,-22.988480,-43.228149,0,132
1991833,08-02-2015 16:50:35,A27514,438,-22.982349,-43.223652,20,108
1994580,08-02-2015 16:51:05,A27514,438,-22.981960,-43.223759,0,127
2001614,08-02-2015 16:53:35,A27514,438,-22.978621,-43.224411,0,76
2004626,08-02-2015 16:54:35,A27514,438,-22.977249,-43.224899,0,65
2007975,08-02-2015 16:55:35,A27514,438,-22.973619,-43.225529,4,104
2010612,08-02-2015 16:56:05,A27514,438,-22.973579,-43.225491,7,123


In [112]:
for row in gps438_A27592_4_44pm_adelante.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    mymapnew.polygon_marker( latlon, fill_color='green', num_sides=4, radius=6)
    
embed_map(mymapnew)